# Activation Checkpointing: A First Principles Deep Dive

Memory is the bottleneck in deep learning. Not compute. Not data. Memory.

This notebook tears apart activation checkpointing from first principles. You will understand exactly why activations dominate memory, how checkpointing trades compute for memory, and when this trade-off makes sense.

---

## The Core Problem

During training you do two things:
1. **Forward pass**: compute outputs layer by layer
2. **Backward pass**: compute gradients using the chain rule

Here's what kills you: backward needs intermediate values from forward. If forward does `y = sin(x)`, backward needs `x` to compute `dy/dx = cos(x)`. PyTorch keeps `x` alive until backward reaches that node. These are "saved tensors" in autograd-speak.

So activation memory accumulates through forward and peaks at the start of backward. You've got this pile of "will need later" tensors sitting in VRAM, waiting.

## What Checkpointing Actually Does

Think of training like hiking a trail:

**Normal training**: You drop breadcrumbs at every step (store all activations). Easy to retrace (backprop), but you're carrying a giant breadcrumb bag (VRAM).

**Checkpointing**: You only drop breadcrumbs at a few checkpoints. On the way back, when you need details between checkpoints, you re-walk that segment.

That's literally it. Save fewer tensors in forward, recompute them on demand during backward. Memory goes down, compute goes up.

## Concrete Example

Suppose your forward is:

```
x --> [f1] --> a --> [f2] --> b --> [f3] --> y
```

**No checkpointing**: autograd saves `a` and `b` so backward can compute gradients.

**Checkpoint after `a`**: you keep `a`, but you don't keep `b`. During backward, when you need `b` to differentiate `f3`, PyTorch reruns `f2(a)` to get it back.

Memory goes down (you didn't store `b`). Compute goes up (you recomputed `b`).

---

## 0. Setup

In [14]:
# 0.1 Environment Check

import torch
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

PyTorch Version: 2.9.0+cu126
CUDA Available: True
CUDA Version: 12.6
GPU: Tesla T4
GPU Memory: 15.83 GB


---

# 1. The Memory Problem

Training deep neural networks hits a wall. That wall is GPU memory.

Four things consume GPU memory during training:
1. **Model parameters** (weights and biases)
2. **Gradients** (same size as parameters)
3. **Optimizer states** (for Adam: 2x parameter size for m and v buffers)
4. **Activations** (intermediate outputs from each layer)

For Adam/AdamW, the first three combined are 4x the model size (params + grads + 2 momentum buffers). This is fixed cost.

Here is the surprise: **activations often dominate anyway**. Not parameters. Not gradients. Not optimizer states. Activations. Why? Because activations scale with batch size. Everything else does not.

In [15]:
# 1.1 Memory Breakdown: Theory, Measurement, and Timing

import torch
import torch.nn as nn

# ===== PART 1: Theoretical Breakdown =====

def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

def bytes_to_mb(b):
    return b / (1024 * 1024)

class DeepNetwork(nn.Module):
    def __init__(self, input_size=1024, hidden_size=1024, num_layers=10):
        super().__init__()
        layers = []
        for i in range(num_layers):
            in_f = input_size if i == 0 else hidden_size
            layers.append(nn.Linear(in_f, hidden_size))
            layers.append(nn.ReLU())
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

model = DeepNetwork(input_size=1024, hidden_size=1024, num_layers=10)
num_params = count_parameters(model)

# Theoretical memory breakdown for a full training setup with Adam
param_mem = num_params * 4  # fp32 parameters
grad_mem = param_mem        # gradients
optimizer_mem = param_mem * 2  # Adam's m and v buffers

print("=" * 60)
print("PART 1: THEORETICAL MEMORY BREAKDOWN")
print("=" * 60)
print(f"Model: {num_params:,} parameters")
print()
print("Fixed costs for training with Adam:")
print(f"  Parameter memory:  {bytes_to_mb(param_mem):.2f} MB")
print(f"  Gradient memory:   {bytes_to_mb(grad_mem):.2f} MB")
print(f"  Optimizer memory:  {bytes_to_mb(optimizer_mem):.2f} MB (m + v)")
print(f"  Total fixed:       {bytes_to_mb(param_mem + grad_mem + optimizer_mem):.2f} MB")
print()
print("Activation memory by batch size (THEORETICAL):")
fixed_cost = param_mem + grad_mem + optimizer_mem
for batch_size in [32, 128, 512]:
    act_mem = batch_size * 1024 * 10 * 4
    total = fixed_cost + act_mem
    act_pct = (act_mem / total) * 100
    print(f"  Batch {batch_size:3d}: activations = {bytes_to_mb(act_mem):6.2f} MB ({act_pct:.1f}% of total)")

# ===== PART 2: Real GPU Measurement =====

print()
print("=" * 60)
print("PART 2: REAL GPU MEASUREMENT")
print("=" * 60)

if torch.cuda.is_available():
    # Wider network to make activations dominate
    class WideNetwork(nn.Module):
        def __init__(self, hidden_size=4096, num_layers=6):
            super().__init__()
            layers = []
            for i in range(num_layers):
                layers.append(nn.Linear(1024 if i == 0 else hidden_size, hidden_size))
                layers.append(nn.ReLU())
            self.layers = nn.Sequential(*layers)
        
        def forward(self, x):
            return self.layers(x)

    model = WideNetwork().cuda()
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print()
    
    for batch_size in [64, 256, 512]:
        torch.cuda.reset_peak_memory_stats()
        torch.cuda.empty_cache()
        
        x = torch.randn(batch_size, 1024).cuda()
        output = model(x)
        loss = output.sum()
        loss.backward()
        
        peak = torch.cuda.max_memory_allocated() / 1e6
        act_estimate = batch_size * 4096 * 6 * 4 / 1e6
        print(f"Batch {batch_size:3d}: Peak = {peak:.1f} MB  (est. activations: {act_estimate:.1f} MB)")
        
        model.zero_grad()
        del x, output, loss
    
    del model
    torch.cuda.empty_cache()
else:
    print("[Run on GPU to see measurements]")

# ===== PART 3: Memory Timeline =====

print()
print("=" * 60)
print("PART 3: MEMORY TIMELINE (When Does Memory Peak?)")
print("=" * 60)

class InstrumentedNetwork(nn.Module):
    def __init__(self, num_layers=5, hidden=4096):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Linear(hidden, hidden) for _ in range(num_layers)
        ])
    
    def forward(self, x, track=False):
        memory_log = []
        for i, layer in enumerate(self.layers):
            x = torch.relu(layer(x))
            if track and torch.cuda.is_available():
                torch.cuda.synchronize()
                memory_log.append(torch.cuda.memory_allocated() / 1e6)
        return x, memory_log

if torch.cuda.is_available():
    model = InstrumentedNetwork(num_layers=8, hidden=4096).cuda()
    x = torch.randn(256, 4096).cuda()
    
    num_params = sum(p.numel() for p in model.parameters())
    print(f"Model: {num_params/1e6:.1f}M params")
    print()
    
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.empty_cache()
    mem_model_only = torch.cuda.memory_allocated() / 1e6
    
    output, fwd_mem = model(x, track=True)
    mem_after_forward = torch.cuda.memory_allocated() / 1e6
    
    loss = output.sum()
    loss.backward()
    peak_memory = torch.cuda.max_memory_allocated() / 1e6
    mem_after_backward = torch.cuda.memory_allocated() / 1e6
    
    model.zero_grad(set_to_none=True)
    mem_after_zero_grad = torch.cuda.memory_allocated() / 1e6
    
    print("Memory Timeline:")
    print("-" * 55)
    print(f"{'Model loaded:':<35} {mem_model_only:>8.1f} MB")
    print()
    print("During forward (activations accumulating):")
    for i, mem in enumerate(fwd_mem):
        delta = mem - (fwd_mem[i-1] if i > 0 else mem_model_only)
        print(f"  After layer {i+1}:{'':<23} {mem:>8.1f} MB  (+{delta:.1f})")
    print()
    print(f"{'After forward (all activations live):':<35} {mem_after_forward:>8.1f} MB")
    print(f"{'PEAK (activations + gradients):':<35} {peak_memory:>8.1f} MB  <-- this kills you")
    print(f"{'After backward:':<35} {mem_after_backward:>8.1f} MB")
    print(f"{'After zero_grad:':<35} {mem_after_zero_grad:>8.1f} MB")
    print()
    
    activation_mem = mem_after_forward - mem_model_only
    print(f"KEY INSIGHT: Activation memory ~{activation_mem:.1f} MB dominates during forward pass")
    print("Memory peaks at start of backward when both activations and gradients exist")
else:
    print("[Run on GPU to see memory timeline]")

PART 1: THEORETICAL MEMORY BREAKDOWN
Model: 10,496,000 parameters

Fixed costs for training with Adam:
  Parameter memory:  40.04 MB
  Gradient memory:   40.04 MB
  Optimizer memory:  80.08 MB (m + v)
  Total fixed:       160.16 MB

Activation memory by batch size (THEORETICAL):
  Batch  32: activations =   1.25 MB (0.8% of total)
  Batch 128: activations =   5.00 MB (3.0% of total)
  Batch 512: activations =  20.00 MB (11.1% of total)

PART 2: REAL GPU MEASUREMENT
GPU: Tesla T4

Batch  64: Peak = 4435.3 MB  (est. activations: 6.3 MB)
Batch 256: Peak = 4442.4 MB  (est. activations: 25.2 MB)
Batch 512: Peak = 4451.8 MB  (est. activations: 50.3 MB)

PART 3: MEMORY TIMELINE (When Does Memory Peak?)
Model: 134.3M params

Memory Timeline:
-------------------------------------------------------
Model loaded:                         4269.3 MB

During forward (activations accumulating):
  After layer 1:                          4273.5 MB  (+4.2)
  After layer 2:                          4277.7

---

# 2. Understanding Activation Checkpointing

Activation checkpointing is a memory-compute trade-off. Save memory by not storing activations. Pay for it by recomputing them during backprop.

To understand this, you need to know what activations are and why backprop needs them.

In [16]:
# 2.1 Why Backprop Needs Activations

import torch

# y = W2 * relu(W1 * x)
# Gradient w.r.t. W1 needs the pre-activation W1*x to compute relu'(W1*x)

torch.manual_seed(42)

W1 = torch.randn(4, 4, requires_grad=True)
W2 = torch.randn(4, 4, requires_grad=True)
x = torch.randn(1, 4)

# Forward pass stores these
z1 = x @ W1.T           # Pre-activation (needed for ReLU grad)
a1 = torch.relu(z1)     # Activation (needed for W2 grad)
y = a1 @ W2.T

print("Stored for backprop:")
print(f"  z1: {z1.shape} - needed to compute relu gradient")
print(f"  a1: {a1.shape} - needed to compute W2 gradient")

loss = y.sum()
loss.backward()

print(f"\nGradients computed:")
print(f"  W1.grad: {W1.grad.shape}")
print(f"  W2.grad: {W2.grad.shape}")
print()
print("The chain rule requires intermediate values.")
print("PyTorch keeps them alive until backward finishes.")
print("This is why activations accumulate through forward pass.")

Stored for backprop:
  z1: torch.Size([1, 4]) - needed to compute relu gradient
  a1: torch.Size([1, 4]) - needed to compute W2 gradient

Gradients computed:
  W1.grad: torch.Size([4, 4])
  W2.grad: torch.Size([4, 4])

The chain rule requires intermediate values.
PyTorch keeps them alive until backward finishes.
This is why activations accumulate through forward pass.


In [17]:
# 2.2 The Core Trade-off: Store vs Recompute

import math

print("Standard Training:")
print("  Forward: x -> a1 -> a2 -> a3 -> loss")
print("           [store] [store] [store]")
print("  Backward: uses stored a1, a2, a3")
print("  Memory: O(n) activations")
print()
print("With Checkpointing:")
print("  Forward: x -> a1 -> a2 -> a3 -> loss")
print("           [save]       [save]")
print("  Backward: recompute a2 from a1, then use")
print("  Memory: O(sqrt(n)) with optimal placement")
print()

num_layers = 100
optimal = int(math.sqrt(num_layers))

print(f"For {num_layers} layers:")
print(f"  Standard: {num_layers} activations stored")
print(f"  Checkpointed: ~{optimal} activations stored (at checkpoint boundaries)")
print(f"  Memory reduction: {num_layers/optimal:.0f}x")
print()
print("Compute analysis:")
print(f"  Without checkpointing: n forward + n backward = 2n")
print(f"  With checkpointing:    n forward + n backward + n recompute = 3n")
print(f"  Compute overhead: ~50% of total training time")
print()
print("KEY: The sqrt(n) rule - optimal checkpointing reduces memory from O(n) to O(sqrt(n))")

Standard Training:
  Forward: x -> a1 -> a2 -> a3 -> loss
           [store] [store] [store]
  Backward: uses stored a1, a2, a3
  Memory: O(n) activations

With Checkpointing:
  Forward: x -> a1 -> a2 -> a3 -> loss
           [save]       [save]
  Backward: recompute a2 from a1, then use
  Memory: O(sqrt(n)) with optimal placement

For 100 layers:
  Standard: 100 activations stored
  Checkpointed: ~10 activations stored (at checkpoint boundaries)
  Memory reduction: 10x

Compute analysis:
  Without checkpointing: n forward + n backward = 2n
  With checkpointing:    n forward + n backward + n recompute = 3n
  Compute overhead: ~50% of total training time

KEY: The sqrt(n) rule - optimal checkpointing reduces memory from O(n) to O(sqrt(n))


---

# 3. PyTorch API - Basic Usage

PyTorch provides `torch.utils.checkpoint.checkpoint()` for activation checkpointing. The basic pattern is simple: wrap the function you want to checkpoint.

In [18]:
# 3.1 Basic Checkpoint: Side-by-Side Comparison

import torch
import torch.nn as nn
from torch.utils.checkpoint import checkpoint

# ===== WITHOUT Checkpointing =====
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(1024, 1024)
        self.layer2 = nn.Linear(1024, 1024)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        return x

# ===== WITH Checkpointing =====
class CheckpointedModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(1024, 1024)
        self.layer2 = nn.Linear(1024, 1024)

    def forward(self, x):
        # Run layer1 normally (so input to checkpointed region requires grad)
        x = torch.relu(self.layer1(x))
        # Checkpoint layer2: activations won't be stored, will be recomputed
        x = checkpoint(lambda t: torch.relu(self.layer2(t)), x, use_reentrant=False)
        return x

if torch.cuda.is_available():
    # Test without checkpointing
    model1 = SimpleModel().cuda()
    x1 = torch.randn(1, 1024).cuda()
    torch.cuda.reset_peak_memory_stats()
    out1 = model1(x1)
    out1.sum().backward()
    mem_no_ckpt = torch.cuda.max_memory_allocated() / 1e6
    del model1, x1, out1
    torch.cuda.empty_cache()
    
    # Test with checkpointing
    model2 = CheckpointedModel().cuda()
    x2 = torch.randn(1, 1024).cuda()
    torch.cuda.reset_peak_memory_stats()
    out2 = model2(x2)
    out2.sum().backward()
    mem_with_ckpt = torch.cuda.max_memory_allocated() / 1e6
    
    print("Basic Checkpoint Comparison (small model)")
    print("-" * 50)
    print(f"WITHOUT Checkpointing: {mem_no_ckpt:.2f} MB")
    print(f"WITH Checkpointing:    {mem_with_ckpt:.2f} MB")
    print()
    print("NOTE: Small model shows minimal difference.")
    print("Savings become significant with deeper models (see Section 5: ResNet50).")
else:
    print("[Run on GPU to see memory comparison]")
    print()
    print("API Usage:")
    print("  checkpoint(fn, *args, use_reentrant=False)")
    print("  - fn: callable to checkpoint")
    print("  - Always use use_reentrant=False (modern API)")

Basic Checkpoint Comparison (small model)
--------------------------------------------------
WITHOUT Checkpointing: 4286.08 MB
WITH Checkpointing:    4277.69 MB

NOTE: Small model shows minimal difference.
Savings become significant with deeper models (see Section 5: ResNet50).


---

# 4. torch.compile and Memory Budget API

torch.compile (PyTorch 2.0+) traces your forward and backward passes into a single joint graph and applies a "min-cut" partitioner that automatically decides which tensors to save and which to recompute.

By default, min-cut prioritizes speed, not memory. The Memory Budget API gives you control over this trade-off.

In [19]:
# 4.1 torch.compile: The Min-Cut Partitioner

import torch
import torch.nn as nn

print("How torch.compile handles activations:")
print("-" * 50)
print()
print("1. TRACING")
print("   torch.compile traces forward AND backward into one graph.")
print("   This lets it see the whole picture.")
print()
print("2. MIN-CUT PARTITIONING")
print("   The graph is split at the optimal points.")
print("   Algorithm minimizes: tensors crossing the cut")
print("   (These are the tensors saved for backward)")
print()
print("3. AUTOMATIC RECOMPUTATION")
print("   Cheap ops (relu, add, mul) are recomputed automatically.")
print("   No user intervention needed.")
print()
print("4. FUSION")
print("   Pointwise ops get fused into kernels.")
print("   Fused ops are fast to recompute.")
print()
print("Result: torch.compile gives you SOME memory savings for FREE,")
print("        plus speed improvements from fusion.")

How torch.compile handles activations:
--------------------------------------------------

1. TRACING
   torch.compile traces forward AND backward into one graph.
   This lets it see the whole picture.

2. MIN-CUT PARTITIONING
   The graph is split at the optimal points.
   Algorithm minimizes: tensors crossing the cut
   (These are the tensors saved for backward)

3. AUTOMATIC RECOMPUTATION
   Cheap ops (relu, add, mul) are recomputed automatically.
   No user intervention needed.

4. FUSION
   Pointwise ops get fused into kernels.
   Fused ops are fast to recompute.

Result: torch.compile gives you SOME memory savings for FREE,
        plus speed improvements from fusion.


In [21]:
# 4.2 Comparing: Eager vs Compile vs Checkpointing

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.checkpoint import checkpoint

class SimpleFFN(nn.Module):
    def __init__(self, dim=2048):
        super().__init__()
        self.fc1 = nn.Linear(dim, dim * 4)
        self.fc2 = nn.Linear(dim * 4, dim)
    
    def forward(self, x):
        x = self.fc1(x)
        x = F.gelu(x)
        x = self.fc2(x)
        return x

class Wrapper(nn.Module):
    """Adds a small stem so the checkpointed region receives activations that require grad."""
    def __init__(self, dim=2048, use_checkpoint=False):
        super().__init__()
        self.stem = nn.Linear(dim, dim)
        self.ffn = SimpleFFN(dim)
        self.use_checkpoint = use_checkpoint

    def forward(self, x):
        x = self.stem(x)
        if self.use_checkpoint:
            x = checkpoint(self.ffn, x, use_reentrant=False)
        else:
            x = self.ffn(x)
        return x

def warmup(fn, zero_grad_fn, x, iters=2):
    for _ in range(iters):
        fn(x).sum().backward()
        zero_grad_fn()

def peak_memory_mb(fn, zero_grad_fn, x):
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    out = fn(x)
    out.sum().backward()
    peak = torch.cuda.max_memory_allocated() / 1e6
    zero_grad_fn()
    return peak

if torch.cuda.is_available():
    dim = 2048
    batch = 64
    seq = 128

    x = torch.randn(batch, seq, dim, device="cuda")

    eager_model = Wrapper(dim, use_checkpoint=False).cuda()
    ckpt_model = Wrapper(dim, use_checkpoint=True).cuda()

    results = {}

    # 1) Eager mode
    warmup(eager_model, lambda: eager_model.zero_grad(set_to_none=True), x)
    results['Eager'] = peak_memory_mb(eager_model, lambda: eager_model.zero_grad(set_to_none=True), x)

    # 2) torch.compile (if available)
    if hasattr(torch, "compile"):
        compiled = torch.compile(eager_model)
        warmup(compiled, lambda: eager_model.zero_grad(set_to_none=True), x)
        results['torch.compile'] = peak_memory_mb(compiled, lambda: eager_model.zero_grad(set_to_none=True), x)
    else:
        print("torch.compile not available in this PyTorch build")

    # 3) Activation checkpointing
    warmup(ckpt_model, lambda: ckpt_model.zero_grad(set_to_none=True), x)
    results['Checkpointing'] = peak_memory_mb(ckpt_model, lambda: ckpt_model.zero_grad(set_to_none=True), x)

    print(f"FFN: dim={dim}, batch={batch}, seq={seq}")
    print("-" * 50)
    for name, mem in results.items():
        print(f"{name:20s}: {mem:8.2f} MB")
    print()
    
    # Find best method
    best = min(results, key=results.get)
    print(f"OBSERVATION: {best} gives best memory efficiency here.")
    print()
    print("WHY torch.compile often wins on simple models:")
    print("  - Kernel fusion eliminates intermediate tensor allocations")
    print("  - Min-cut partitioner automatically recomputes cheap ops")
    print("  - Manual checkpointing has overhead without fusion benefits")
    print()
    print("For deeper models (see Section 5: ResNet50), manual checkpointing")
    print("provides more control and can achieve greater savings.")
else:
    print("[Run on GPU for comparison]")

FFN: dim=2048, batch=64, seq=128
--------------------------------------------------
Eager               :  2352.62 MB
torch.compile       :  1933.14 MB
Checkpointing       :  2201.57 MB

OBSERVATION: torch.compile gives best memory efficiency here.

WHY torch.compile often wins on simple models:
  - Kernel fusion eliminates intermediate tensor allocations
  - Min-cut partitioner automatically recomputes cheap ops
  - Manual checkpointing has overhead without fusion benefits

For deeper models (see Section 5: ResNet50), manual checkpointing
provides more control and can achieve greater savings.


### What Gets Recomputed at Each Budget Level?

Based on real transformer results:

| Budget | Recomputes | Saves | Memory |
|--------|------------|-------|--------|
| 1.0 (default) | Nothing extra | Everything | 100% |
| 0.7 | Pointwise ops (gelu, add, mul) | Matmuls, attention | ~85% |
| 0.5 | Pointwise + some matmuls | Attention (most expensive) | ~50% |
| 0.3 | Pointwise + most matmuls | Only attention | ~35% |
| 0.0 | Everything (like full AC) | Almost nothing | Minimum |

**Key insight:** 50% memory reduction by recomputing only pointwise ops. Attention is expensive. Recompute it last.

---

# 4b. External Libraries Reference

Beyond PyTorch's built-in checkpointing, external libraries offer additional features.

In [22]:
# External Libraries: DeepSpeed and FairScale

print("""
DEEPSPEED ACTIVATION CHECKPOINTING
==================================
pip install deepspeed

Key features:
- Automatic checkpoint placement
- CPU offloading for extreme memory savings (offload activations to RAM!)
- Integrated with ZeRO optimizer stages
- Best for: very large models (billions of parameters)

Config in ds_config.json:
{
    "activation_checkpointing": {
        "partition_activations": true,
        "contiguous_memory_optimization": true,
        "cpu_checkpointing": true
    }
}


FAIRSCALE CHECKPOINT_WRAPPER
============================
pip install fairscale

Key features:
- Clean API: wrap modules directly with checkpoint_wrapper()
- Works well with FSDP (Fully Sharded Data Parallel)
- Simpler than DeepSpeed for many use cases
- Best for: medium-scale training

Usage:
from fairscale.nn.checkpoint import checkpoint_wrapper

layer = nn.Sequential(nn.Linear(1024, 1024), nn.ReLU())
checkpointed_layer = checkpoint_wrapper(layer)


TOOL SELECTION GUIDE
====================
- PyTorch native: Best for most cases, well-maintained
- DeepSpeed: Very large models, need CPU offloading
- FairScale: Using FSDP, prefer simpler API
""")


DEEPSPEED ACTIVATION CHECKPOINTING
pip install deepspeed

Key features:
- Automatic checkpoint placement
- CPU offloading for extreme memory savings (offload activations to RAM!)
- Integrated with ZeRO optimizer stages
- Best for: very large models (billions of parameters)

Config in ds_config.json:
{
    "activation_checkpointing": {
        "partition_activations": true,
        "contiguous_memory_optimization": true,
        "cpu_checkpointing": true
    }
}


FAIRSCALE CHECKPOINT_WRAPPER
pip install fairscale

Key features:
- Clean API: wrap modules directly with checkpoint_wrapper()
- Works well with FSDP (Fully Sharded Data Parallel)
- Simpler than DeepSpeed for many use cases
- Best for: medium-scale training

Usage:
from fairscale.nn.checkpoint import checkpoint_wrapper

layer = nn.Sequential(nn.Linear(1024, 1024), nn.ReLU())
checkpointed_layer = checkpoint_wrapper(layer)


TOOL SELECTION GUIDE
- PyTorch native: Best for most cases, well-maintained
- DeepSpeed: Very large mode

---

# 5. Case Study: Activation Checkpointing in ResNet50

ResNet50 is deep enough to benefit from checkpointing. It has 4 stages with multiple bottleneck blocks. Each block stores feature maps for backward.

This section shows how to apply checkpointing to a real production model. You will see:
1. How to modify torchvision's ResNet50
2. Memory comparison across checkpointing strategies
3. A complete training loop with checkpointing

In [23]:
# 5.1 ResNet50 Architecture Overview

import torch
import torch.nn as nn

try:
    import torchvision.models as models
except Exception as e:
    models = None
    print("torchvision is required for the ResNet50 case study.")
    print(f"Details: {type(e).__name__}: {e}")

if models is not None:
    # Load ResNet50 without downloading weights
    try:
        resnet50 = models.resnet50(weights=None)
    except TypeError:
        # Older torchvision
        resnet50 = models.resnet50(pretrained=False)

    print("ResNet50 Structure:")
    print("-" * 60)
    print(f"conv1:  1 conv layer")
    print(f"layer1: {len(resnet50.layer1)} Bottleneck blocks (64 -> 256 channels)")
    print(f"layer2: {len(resnet50.layer2)} Bottleneck blocks (128 -> 512 channels)")
    print(f"layer3: {len(resnet50.layer3)} Bottleneck blocks (256 -> 1024 channels)")
    print(f"layer4: {len(resnet50.layer4)} Bottleneck blocks (512 -> 2048 channels)")
    print(f"fc:     1 linear layer")
    print()
    print(
        f"Total Bottleneck blocks: {len(resnet50.layer1) + len(resnet50.layer2) + len(resnet50.layer3) + len(resnet50.layer4)}"
    )
    print()
    print("Each Bottleneck block has 3 conv layers + skip connection.")
    print("Feature maps grow larger in early layers, then shrink spatially.")
    print("layer3 often has the largest activation memory (1024 channels, moderate spatial size).")

ResNet50 Structure:
------------------------------------------------------------
conv1:  1 conv layer
layer1: 3 Bottleneck blocks (64 -> 256 channels)
layer2: 4 Bottleneck blocks (128 -> 512 channels)
layer3: 6 Bottleneck blocks (256 -> 1024 channels)
layer4: 3 Bottleneck blocks (512 -> 2048 channels)
fc:     1 linear layer

Total Bottleneck blocks: 16

Each Bottleneck block has 3 conv layers + skip connection.
Feature maps grow larger in early layers, then shrink spatially.
layer3 often has the largest activation memory (1024 channels, moderate spatial size).


In [24]:
# 5.2 ResNet50 with Checkpointing: Three Strategies

import torch
import torch.nn as nn
from torch.utils.checkpoint import checkpoint

try:
    import torchvision.models as models
except Exception as e:
    models = None
    ResNet50Checkpointed = None
    print("torchvision is required for ResNet50Checkpointed.")
    print(f"Details: {type(e).__name__}: {e}")

if models is not None:
    class ResNet50Checkpointed(nn.Module):
        """ResNet50 with configurable checkpointing strategies."""

        def __init__(self, num_classes=1000, checkpoint_strategy='none'):
            """
            Args:
                checkpoint_strategy: 'none', 'per_stage', 'per_block', or 'aggressive'
            """
            super().__init__()

            # Load base ResNet50 without downloading weights
            try:
                base = models.resnet50(weights=None)
            except TypeError:
                base = models.resnet50(pretrained=False)

            # Copy all layers
            self.conv1 = base.conv1
            self.bn1 = base.bn1
            self.relu = base.relu
            self.maxpool = base.maxpool
            self.layer1 = base.layer1
            self.layer2 = base.layer2
            self.layer3 = base.layer3
            self.layer4 = base.layer4
            self.avgpool = base.avgpool
            self.fc = nn.Linear(2048, num_classes)

            self.checkpoint_strategy = checkpoint_strategy

        def _forward_stage(self, stage, x):
            """Forward through a stage (layer1, layer2, etc.)."""
            for block in stage:
                x = block(x)
            return x

        def forward(self, x):
            # Stem
            x = self.conv1(x)
            x = self.bn1(x)
            x = self.relu(x)
            x = self.maxpool(x)

            # NOTE: ResNet blocks include BatchNorm, which updates running stats in train mode.
            # Checkpointing recomputes forward in backward, which can update BN stats twice.
            # For strict parity, consider freezing BN stats (eval for BN) or using GroupNorm.

            if self.checkpoint_strategy == 'none':
                x = self.layer1(x)
                x = self.layer2(x)
                x = self.layer3(x)
                x = self.layer4(x)

            elif self.checkpoint_strategy == 'per_stage':
                x = checkpoint(lambda t: self._forward_stage(self.layer1, t), x, use_reentrant=False)
                x = checkpoint(lambda t: self._forward_stage(self.layer2, t), x, use_reentrant=False)
                x = checkpoint(lambda t: self._forward_stage(self.layer3, t), x, use_reentrant=False)
                x = checkpoint(lambda t: self._forward_stage(self.layer4, t), x, use_reentrant=False)

            elif self.checkpoint_strategy == 'per_block':
                for block in self.layer1:
                    x = checkpoint(block, x, use_reentrant=False)
                for block in self.layer2:
                    x = checkpoint(block, x, use_reentrant=False)
                for block in self.layer3:
                    x = checkpoint(block, x, use_reentrant=False)
                for block in self.layer4:
                    x = checkpoint(block, x, use_reentrant=False)

            elif self.checkpoint_strategy == 'aggressive':
                x = self.layer1(x)
                x = self.layer2(x)
                for block in self.layer3:
                    x = checkpoint(block, x, use_reentrant=False)
                for block in self.layer4:
                    x = checkpoint(block, x, use_reentrant=False)
            else:
                raise ValueError(f"Unknown checkpoint_strategy: {self.checkpoint_strategy}")

            # Head
            x = self.avgpool(x)
            x = torch.flatten(x, 1)
            x = self.fc(x)
            return x

    print("ResNet50Checkpointed created with 4 strategies:")
    print("-" * 60)
    print("'none':       No checkpointing (baseline)")
    print("'per_stage':  Checkpoint each of the 4 stages")
    print("'per_block':  Checkpoint each of the 16 bottleneck blocks")
    print("'aggressive': Only checkpoint layer3 and layer4 (best ROI)")

ResNet50Checkpointed created with 4 strategies:
------------------------------------------------------------
'none':       No checkpointing (baseline)
'per_stage':  Checkpoint each of the 4 stages
'per_block':  Checkpoint each of the 16 bottleneck blocks
'aggressive': Only checkpoint layer3 and layer4 (best ROI)


In [25]:
# 5.3 Memory Comparison: ResNet50 Strategies

import torch
import torch.nn as nn
import time

def benchmark_resnet50(strategy, batch_size=32, image_size=224, num_iters=5):
    """Benchmark memory and time for a ResNet50 checkpointing strategy."""
    if ResNet50Checkpointed is None:
        raise RuntimeError("ResNet50Checkpointed is not available (torchvision import likely failed).")

    model = ResNet50Checkpointed(num_classes=1000, checkpoint_strategy=strategy).cuda()
    model.train()

    x = torch.randn(batch_size, 3, image_size, image_size, device="cuda")
    target = torch.randint(0, 1000, (batch_size,), device="cuda")
    criterion = nn.CrossEntropyLoss()

    # Warmup
    for _ in range(2):
        out = model(x)
        loss = criterion(out, target)
        loss.backward()
        model.zero_grad(set_to_none=True)

    # Measure memory + time
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.synchronize()

    start = time.time()
    for _ in range(num_iters):
        out = model(x)
        loss = criterion(out, target)
        loss.backward()
        model.zero_grad(set_to_none=True)
    torch.cuda.synchronize()

    elapsed = (time.time() - start) / num_iters * 1000  # ms/iter
    peak_mem = torch.cuda.max_memory_allocated() / 1e6  # MB

    del model, x, target
    torch.cuda.empty_cache()

    return peak_mem, elapsed

if torch.cuda.is_available() and ResNet50Checkpointed is not None:
    batch_size = 32
    image_size = 224

    print("ResNet50 Checkpointing Comparison")
    print(f"Batch size: {batch_size}, Image size: {image_size}x{image_size}")
    print("-" * 70)
    print(f"{'Strategy':<15} {'Peak Memory (MB)':<20} {'Time (ms)':<15} {'Mem Savings':<15}")
    print("-" * 70)

    strategies = ['none', 'aggressive', 'per_stage', 'per_block']
    results = {}

    for strategy in strategies:
        mem, time_ms = benchmark_resnet50(strategy, batch_size, image_size)
        results[strategy] = (mem, time_ms)

    baseline_mem = results['none'][0]
    for strategy in strategies:
        mem, time_ms = results[strategy]
        savings = (1 - mem / baseline_mem) * 100
        print(f"{strategy:<15} {mem:<20.2f} {time_ms:<15.2f} {savings:>6.1f}%")

    print("-" * 70)
    print("\nObservations:")
    print("- 'aggressive' gives best memory/speed balance (checkpoints only deep layers)")
    print("- 'per_block' gives maximum memory savings but highest overhead")
    print("- 'per_stage' is a middle ground")
else:
    print("[Run on GPU and ensure torchvision is installed for ResNet50 benchmarks]")

ResNet50 Checkpointing Comparison
Batch size: 32, Image size: 224x224
----------------------------------------------------------------------
Strategy        Peak Memory (MB)     Time (ms)       Mem Savings    
----------------------------------------------------------------------
none            4143.20              279.37             0.0%
aggressive      3737.85              318.93             9.8%
per_stage       2873.71              369.58            30.6%
per_block       2468.50              368.85            40.4%
----------------------------------------------------------------------

Observations:
- 'aggressive' gives best memory/speed balance (checkpoints only deep layers)
- 'per_block' gives maximum memory savings but highest overhead
- 'per_stage' is a middle ground


In [26]:
# 5.4 Maximum Batch Size: ResNet50

import torch
import torch.nn as nn

def find_max_batch_resnet50(strategy, start=16, max_batch=256):
    """Find maximum batch size before OOM for ResNet50 (CUDA only)."""
    if not torch.cuda.is_available():
        raise RuntimeError("CUDA is required for find_max_batch_resnet50().")
    if ResNet50Checkpointed is None:
        raise RuntimeError("ResNet50Checkpointed is not available (torchvision import likely failed).")

    start = max(1, int(start))
    max_batch = max(start, int(max_batch))

    def can_run(batch_size):
        model = None
        x = None
        target = None
        out = None
        loss = None
        try:
            torch.cuda.empty_cache()
            model = ResNet50Checkpointed(checkpoint_strategy=strategy).cuda()
            model.train()
            x = torch.randn(batch_size, 3, 224, 224, device="cuda")
            target = torch.randint(0, 1000, (batch_size,), device="cuda")

            out = model(x)
            loss = nn.CrossEntropyLoss()(out, target)
            loss.backward()
            return True
        except RuntimeError as e:
            if "out of memory" in str(e).lower():
                return False
            raise
        finally:
            try:
                if model is not None:
                    model.zero_grad(set_to_none=True)
            except Exception:
                pass
            del model, x, target, out, loss
            torch.cuda.empty_cache()

    # Exponential search to find an upper bound, then binary search.
    max_working = 0
    batch = start
    while batch <= max_batch and can_run(batch):
        max_working = batch
        batch *= 2

    low = max_working + 1
    high = min(batch, max_batch)

    while low <= high:
        mid = (low + high) // 2
        if can_run(mid):
            max_working = mid
            low = mid + 1
        else:
            high = mid - 1

    return max_working

if torch.cuda.is_available() and ResNet50Checkpointed is not None:
    print("Finding maximum batch size for ResNet50 (224x224 images)")
    print("-" * 50)

    strategies = ['none', 'aggressive', 'per_block']
    baseline = None

    for strategy in strategies:
        max_batch = find_max_batch_resnet50(strategy)
        if baseline is None:
            baseline = max_batch
        improvement = (max_batch / baseline) if baseline else float('inf')
        print(f"{strategy:<15}: max batch = {max_batch:>3d} ({improvement:.2f}x)")

    print("-" * 50)
    print("\nWith checkpointing, you can often fit larger batches.")
else:
    print("[Run on GPU and ensure torchvision is installed to find max batch sizes]")

Finding maximum batch size for ResNet50 (224x224 images)
--------------------------------------------------
none           : max batch = 164 (1.00x)
aggressive     : max batch = 192 (1.17x)
per_block      : max batch = 256 (1.56x)
--------------------------------------------------

With checkpointing, you can often fit larger batches.


In [27]:
# 5.5 Complete Training Loop: ResNet50 with Checkpointing

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

def train_resnet50_with_checkpointing(
    checkpoint_strategy='aggressive',
    batch_size=32,
    num_epochs=3,
    num_samples=256,  # Small for demo
    use_amp=True,
):
    """Complete training loop with checkpointing and mixed precision."""

    if ResNet50Checkpointed is None:
        raise RuntimeError("ResNet50Checkpointed is not available (torchvision import likely failed).")

    # Create model
    model = ResNet50Checkpointed(
        num_classes=10,  # Simplified for demo
        checkpoint_strategy=checkpoint_strategy,
    ).cuda()

    # Create synthetic dataset
    X = torch.randn(num_samples, 3, 224, 224)
    y = torch.randint(0, 10, (num_samples,))
    dataset = TensorDataset(X, y)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Optimizer and loss
    optimizer = optim.AdamW(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()
    scaler = torch.cuda.amp.GradScaler() if (use_amp and torch.cuda.is_available()) else None

    print(f"Training ResNet50 with '{checkpoint_strategy}' checkpointing")
    print(f"Batch size: {batch_size}, Mixed precision: {use_amp and scaler is not None}")
    print("-" * 50)

    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        num_batches = 0

        torch.cuda.reset_peak_memory_stats()

        for batch_x, batch_y in loader:
            batch_x = batch_x.cuda()
            batch_y = batch_y.cuda()

            optimizer.zero_grad(set_to_none=True)

            if scaler is not None:
                with torch.cuda.amp.autocast():
                    outputs = model(batch_x)
                    loss = criterion(outputs, batch_y)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

            epoch_loss += loss.item()
            num_batches += 1

        peak_mem = torch.cuda.max_memory_allocated() / 1e6
        avg_loss = epoch_loss / num_batches
        print(f"Epoch {epoch+1}/{num_epochs}: loss = {avg_loss:.4f}, peak memory = {peak_mem:.2f} MB")

    print("-" * 50)
    print("Training complete!")
    return model

if torch.cuda.is_available() and ResNet50Checkpointed is not None:
    model = train_resnet50_with_checkpointing(
        checkpoint_strategy='aggressive',
        batch_size=32,
        num_epochs=3,
        use_amp=True,
    )
else:
    print("[Run on GPU and ensure torchvision is installed for training demo]")

/tmp/ipython-input-1592307309.py:35: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() if (use_amp and torch.cuda.is_available()) else None
/tmp/ipython-input-1592307309.py:55: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Training ResNet50 with 'aggressive' checkpointing
Batch size: 32, Mixed precision: True
--------------------------------------------------
Epoch 1/3: loss = 2.4102, peak memory = 2739.79 MB
Epoch 2/3: loss = 1.9871, peak memory = 2739.79 MB
Epoch 3/3: loss = 1.5362, peak memory = 2739.79 MB
--------------------------------------------------
Training complete!


In [28]:
# 5.6 Alternative: Using torchvision's Built-in Support

import torch
from torch.utils.checkpoint import checkpoint_sequential

try:
    import torchvision.models as models
except Exception as e:
    models = None
    print("torchvision is required for this alternative ResNet50 example.")
    print(f"Details: {type(e).__name__}: {e}")


def _checkpoint_sequential(mod, segments, x):
    """checkpoint_sequential signature varies across PyTorch versions."""
    try:
        return checkpoint_sequential(mod, segments, x, use_reentrant=False)
    except TypeError:
        return checkpoint_sequential(mod, segments, x)


def resnet50_with_sequential_checkpoint(num_segments=4):
    """Monkey-patch torchvision ResNet50 to checkpoint its stage blocks."""
    if models is None:
        raise RuntimeError("torchvision is not available")

    try:
        model = models.resnet50(weights=None)
    except TypeError:
        model = models.resnet50(pretrained=False)

    def checkpointed_forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        # Checkpoint each stage with user-controlled granularity.
        x = _checkpoint_sequential(self.layer1, min(num_segments, len(self.layer1)), x)
        x = _checkpoint_sequential(self.layer2, min(num_segments, len(self.layer2)), x)
        x = _checkpoint_sequential(self.layer3, min(num_segments, len(self.layer3)), x)
        x = _checkpoint_sequential(self.layer4, min(num_segments, len(self.layer4)), x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

    import types

    model.forward = types.MethodType(checkpointed_forward, model)
    return model

# Test it
if models is not None:
    model = resnet50_with_sequential_checkpoint(num_segments=4)
    print("Alternative approach: Monkey-patch torchvision ResNet50")
    print("-" * 60)
    print("Pros:")
    print("  - Uses official torchvision model")
    print("  - Simple implementation")
    print("  - Adjustable granularity via num_segments")
    print()
    print("Cons:")
    print("  - Less control over which blocks to checkpoint")
    print("  - Harder to switch strategies dynamically")
    print()
    print("Use the custom class (ResNet50Checkpointed) for flexible experiments.")

Alternative approach: Monkey-patch torchvision ResNet50
------------------------------------------------------------
Pros:
  - Uses official torchvision model
  - Simple implementation
  - Adjustable granularity via num_segments

Cons:
  - Less control over which blocks to checkpoint
  - Harder to switch strategies dynamically

Use the custom class (ResNet50Checkpointed) for flexible experiments.


### Impressions/Conclusions (5: ResNet50 Case Study)

ResNet50 is a perfect testbed for checkpointing because:
- 16 bottleneck blocks across 4 stages
- Deep enough to benefit, not so deep it is impractical
- Widely used in production

Key findings:

**Strategy Selection:**
- `aggressive` (checkpoint layer3+layer4): Best ROI. These layers have the most parameters and largest feature maps.
- `per_block`: Maximum memory savings (~40-50%), but highest compute overhead (~30-40%).
- `per_stage`: Middle ground. Good for quick wins.

**Practical Impact:**
- 1.5-2x larger batch sizes possible
- Combined with mixed precision: up to 3x improvement

**BatchNorm Caveat:**

ResNet blocks use BatchNorm. Checkpointing recomputes forward in backward, which can update BN running stats twice in `train()` mode. If you need strict parity, consider freezing BN stats (set BN layers to eval) or using GroupNorm.

**Production Recommendations:**
1. Start with `aggressive` strategy
2. Combine with mixed precision (AMP)
3. Profile before and after
4. If still OOM, move to `per_block`

The ResNet50Checkpointed class is a solid starting point. Copy it into your codebase and adapt it to your training setup and normalization strategy.

---

# 6. Conclusion

Activation checkpointing is a memory-compute trade-off. It trades extra forward passes for reduced memory usage. The trade-off is usually worth it.

In [29]:
# 6.1 Key Takeaways

print("""
KEY TAKEAWAYS
=============

1. ACTIVATIONS DOMINATE MEMORY
   - Not parameters, not gradients
   - Scales with batch_size * depth * hidden_size
   - Peak memory occurs at start of backward pass

2. CHECKPOINTING TRADES COMPUTE FOR MEMORY
   - Save some activations (checkpoints)
   - Recompute others during backward pass
   - Typical: 30-50% memory savings, ~50% compute overhead

3. THE SQRT(N) RULE
   - Optimal checkpointing: O(sqrt(n)) memory for n layers
   - 100 layers -> ~10 checkpoints -> 10x memory reduction

4. PRACTICAL PATTERNS
   - checkpoint(): Single modules/functions
   - checkpoint_sequential(): Sequential models
   - Always use use_reentrant=False

5. ADVANCED: SELECTIVE ACTIVATION CHECKPOINT (SAC)
   - Fine-grained control: choose what to save vs recompute
   - Policy functions: MUST_SAVE for expensive ops, PREFER_RECOMPUTE for cheap
   - Sweet spot: save matmuls, recompute pointwise ops
   - Use aten.<op>.default for correct op matching

6. ADVANCED: MEMORY BUDGET API (torch.compile)
   - One line: torch._dynamo.config.activation_memory_budget = 0.5
   - Automatic pareto-optimal recomputation strategies
   - Budget 0 = full AC, Budget 1 = default compile

7. COMBINE WITH OTHER TECHNIQUES
   - Mixed precision: 2x memory from fp16
   - Gradient accumulation: Effective larger batches
   - DDP: Scale across GPUs
   - Together: Train 4-5x larger models
""")


KEY TAKEAWAYS

1. ACTIVATIONS DOMINATE MEMORY
   - Not parameters, not gradients
   - Scales with batch_size * depth * hidden_size
   - Peak memory occurs at start of backward pass

2. CHECKPOINTING TRADES COMPUTE FOR MEMORY
   - Save some activations (checkpoints)
   - Recompute others during backward pass
   - Typical: 30-50% memory savings, ~50% compute overhead

3. THE SQRT(N) RULE
   - Optimal checkpointing: O(sqrt(n)) memory for n layers
   - 100 layers -> ~10 checkpoints -> 10x memory reduction

4. PRACTICAL PATTERNS
   - checkpoint(): Single modules/functions
   - checkpoint_sequential(): Sequential models
   - Always use use_reentrant=False

5. ADVANCED: SELECTIVE ACTIVATION CHECKPOINT (SAC)
   - Fine-grained control: choose what to save vs recompute
   - Policy functions: MUST_SAVE for expensive ops, PREFER_RECOMPUTE for cheap
   - Sweet spot: save matmuls, recompute pointwise ops
   - Use aten.<op>.default for correct op matching

6. ADVANCED: MEMORY BUDGET API (torch.compi

In [30]:
# 6.2 Decision Framework: When to Use Checkpointing

print("""
WHEN TO USE ACTIVATION CHECKPOINTING
====================================

USE IT WHEN:
- You are hitting OOM errors
- You want larger batch sizes
- Your model has 10+ layers
- Training time is less critical than memory
- You are training transformers or deep CNNs

SKIP IT WHEN:
- Model fits comfortably in memory
- Training time is the bottleneck (not memory)
- Model is shallow (< 5 layers)
- You need maximum training speed

QUICK DECISION TREE:

  OOM Error?
     |
     v
  YES --> Use checkpointing
     |
     v
  Want larger batches?
     |
     v
  YES --> Use checkpointing
     |
     v
  Model > 10 layers?
     |
     v
  YES --> Consider checkpointing
     |
     v
  NO --> Probably skip it
""")


WHEN TO USE ACTIVATION CHECKPOINTING

USE IT WHEN:
- You are hitting OOM errors
- You want larger batch sizes
- Your model has 10+ layers
- Training time is less critical than memory
- You are training transformers or deep CNNs

SKIP IT WHEN:
- Model fits comfortably in memory
- Training time is the bottleneck (not memory)
- Model is shallow (< 5 layers)
- You need maximum training speed

QUICK DECISION TREE:

  OOM Error?
     |
     v
  YES --> Use checkpointing
     |
     v
  Want larger batches?
     |
     v
  YES --> Use checkpointing
     |
     v
  Model > 10 layers?
     |
     v
  YES --> Consider checkpointing
     |
     v
  NO --> Probably skip it



### Final Impressions

Activation checkpointing is not magic. It is a simple trade-off executed well.

You now understand:
- Why activations dominate memory (batch x depth x hidden)
- How checkpointing works (recompute instead of store)
- When to use it (memory-bound, deep models)
- Basic implementation (checkpoint, checkpoint_sequential, use_reentrant=False)
- Advanced control with SAC (choose exactly what to save)
- Automatic optimization with Memory Budget API (one config line)

The landscape of techniques:
- **Eager**: Maximum speed, maximum memory
- **torch.compile**: Free speedups, some automatic memory savings
- **Memory Budget API**: Tunable compile-time optimization (0 to 1)
- **Selective AC**: Manual control over save/recompute decisions
- **Standard AC**: Maximum memory savings, ~50% compute overhead

Start simple. Add complexity only when needed. Measure everything.

Every large language model uses some form of activation checkpointing. Now you know exactly how it works and when to use each variant.

Go train something bigger.